In [74]:
!which python

/Users/apabook/Desktop/ds-deployment/venv/bin/python


In [75]:
import json
import requests

# Complete Dataspace Flow


## Consts
Configuration of constant variables used throughout the notebook, including agent URLs and participant IDs.


In [ ]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_provider_dsp_endpoint = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Dataspace setup
Authority is created and Consumer Agent and Provider Agent are registered in the Authority. Then, both participants onboard themselves in the system.


### Current Participants Information


In [77]:
url = data_space_provider + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:provider",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "saved_at": "2026-02-06T13:26:50.667174",
    "last_interaction": "2026-02-06T13:26:50.670355",
    "is_me": true
}


In [78]:
url = data_space_consumer + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:consumer",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "saved_at": "2026-02-06T13:28:35.441432",
    "last_interaction": "2026-02-06T13:28:35.441465",
    "is_me": true
}


### Provider and Consumer IDs
Credential exchange and token retrieval for API calls.


In [79]:
url = data_space_provider + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_consumer_participant_id:
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

In [80]:
url = data_space_consumer + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_provider_participant_id:
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

In [81]:
print("Provider DID: {}\n".format(data_space_provider_participant_id))
print("Provider token: {}\n".format(data_space_provider_participant_token))
print("Consumer DID: {}\n".format(data_space_consumer_participant_id))
print("Consumer token: {}".format(data_space_consumer_participant_token))

Provider DID: did:jwk:provider

Provider token: oTqd5VoX7Xk37hJcSSlvpyXCy2NtvWqwH7WWCJ7tzQg

Consumer DID: did:jwk:consumer

Consumer token: oTqd5VoX7Xk37hJcSSlvpyXCy2NtvWqwH7WWCJ7tzQg


## Create Dataset, Distributions, Policies and Connector Instances
Provider is going to create a catalog, a data service, a dataset and a distribution to be offered to the consumer. The consumer will then discover the offer and accept it.


In [82]:
url = data_space_provider + "/api/v1/catalog-agent/catalogs/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "dctConformsTo": null,
    "dctCreator": null,
    "dctIdentifier": "urn:catalog:0e3162bb-e2ee-488d-a0c5-4d9bc3d6f70c",
    "dctIssued": "2026-02-06T13:31:00.119568Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainCatalog": true,
    "dspaceParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "foafHomePage": null,
    "id": "urn:catalog:0e3162bb-e2ee-488d-a0c5-4d9bc3d6f70c"
}


In [83]:
url = data_space_provider + "/api/v1/catalog-agent/data-services/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_service_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:0e3162bb-e2ee-488d-a0c5-4d9bc3d6f70c",
    "dcatEndpointDescription": null,
    "dcatEndpointUrl": "http://127.0.0.1:1200/dsp/current",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:data-service:5a94c5cd-4f65-4e9a-baac-4f64110f5732",
    "dctIssued": "2026-02-06T13:31:00.222611Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainDataService": true,
    "id": "urn:data-service:5a94c5cd-4f65-4e9a-baac-4f64110f5732"
}


In [84]:
url = data_space_provider + "/api/v1/catalog-agent/datasets"
payload = {
    "dctTitle": "My dataspace dataset",
    "catalogId": catalog_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:0e3162bb-e2ee-488d-a0c5-4d9bc3d6f70c",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:dataset:9cc999e7-6294-4d5d-94ae-18479a28b7a3",
    "dctIssued": "2026-02-06T13:50:17.527562Z",
    "dctModified": null,
    "dctTitle": "My dataspace dataset",
    "id": "urn:dataset:9cc999e7-6294-4d5d-94ae-18479a28b7a3"
}


In [85]:
url = data_space_provider + "/api/v1/catalog-agent/distributions"
payload = {
    "dctTitle": "Distribution of my dataset",
    "dctDescription": "This is the distribution of my dataset",
    "dcatAccessService": data_service_id,
    "datasetId": dataset_id,
    "dctFormats": "http+push"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "datasetId": "urn:dataset:9cc999e7-6294-4d5d-94ae-18479a28b7a3",
    "dcatAccessService": "urn:data-service:5a94c5cd-4f65-4e9a-baac-4f64110f5732",
    "dctDescription": "This is the distribution of my dataset",
    "dctFormat": "Http+Push",
    "dctIssued": "2026-02-06T13:50:17.951278Z",
    "dctModified": null,
    "dctTitle": "Distribution of my dataset",
    "id": "urn:distribution:b4b96cad-248a-42c3-8111-f29d8082510e"
}


## Create policies


In [86]:
url = data_space_provider + "/api/v1/catalog-agent/odrl-policies"
payload = {
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "rightOperand": "user",
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq"
                    }
                ]
            }

        ],
        "obligation": [],
        "prohibition": []
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-06T13:50:18.827261Z",
    "entity": "urn:dataset:9cc999e7-6294-4d5d-94ae-18479a28b7a3",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:e76451c4-6499-4229-a3fa-d272fdc5b2fe",
    "instantiationParameters": null,
    "odrlOffer": {
        "obligation": [],
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            }
        ],
        "prohibition": []
    },
    "sourceTemplateId": null,
    "sourceTemplateVersion": null
}


## Create and configure connector to final system



In [93]:
url = data_space_provider + "/api/v1/connector/templates"
payload = {
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "*"
        }
    },
    "parameters": [
        {
            "paramType": "STRING",
            "name": "ACCESS_URL",
            "title": "Url",
            "required": True
        }
    ]
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
connector_name = response_as_json["name"]
connector_version = response_as_json["version"]
print(json.dumps(response_as_json, indent=4))

{
    "name": "urn:connector-template:349d28a1-fad8-4211-9a4c-6cc45117349d",
    "author": "admin",
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-07T11:43:42.677055Z",
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "*",
            "headers": null,
            "bodyTemplate": null
        }
    },
    "parameters": [
        {
            "name": "ACCESS_URL",
            "title": "Url",
            "description": null,
            "paramType": "STRING",
            "required": true,
            "defaultValue": null,
            "autoFilled": false,
            "autoFilledType": null
        }
    ]
}


In [94]:
url = data_space_provider + "/api/v1/connector/instances"
payload = {
    "templateName": connector_name,
    "templateVersion": connector_version,
    "distributionId": distribution_id,
    "metadata": {
        "description": "asasdasddas",
        "ownerId": "asdas"
    },
    "dryRun": False,
    "parameters": {
        "ACCESS_URL": "https://blabla:123/data/access/bla"
    }
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "id": "urn:connector-instance:f1750a72-2523-42ca-8c17-453f3bbd1190",
    "name": "urn:connector-template:349d28a1-fad8-4211-9a4c-6cc45117349d",
    "author": null,
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-07T11:43:44.267697Z",
    "authenticationConfig": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "https://blabla:123/data/access/bla",
            "method": "*",
            "headers": null,
            "bodyTemplate": null
        }
    },
    "distributionId": "urn:distribution:b4b96cad-248a-42c3-8111-f29d8082510e"
}


## Consumer discovers the provider catalog and finds the offer


In [87]:
url = data_space_consumer + "/dsp/current/catalog/rpc/setup-catalog-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "filter": []
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["response"]["dataset"][0]["hasPolicy"][0]["@id"]
target_id = response_as_json["response"]["dataset"][0]["@id"]
data_space_provider_dsp_endpoint = response_as_json["response"]["service"]["endpointURL"]
response_as_json


{'request': {'associatedAgentPeer': 'did:jwk:provider', 'filter': []},
 'response': {'@context': ['https://w3id.org/dspace/2025/1/context.jsonld'],
  '@type': 'Catalog',
  '@id': 'urn:catalog:0e3162bb-e2ee-488d-a0c5-4d9bc3d6f70c',
  'identifier': 'urn:catalog:0e3162bb-e2ee-488d-a0c5-4d9bc3d6f70c',
  'issued': '2026-02-06T13:31:00.119568',
  'catalog': [],
  'dataset': [{'@type': 'Dataset',
    '@id': 'urn:dataset:b333014d-ee5b-435a-a065-3e0117ea35c0',
    'identifier': 'urn:dataset:b333014d-ee5b-435a-a065-3e0117ea35c0',
    'issued': '2026-02-06T13:37:11.324329',
    'title': 'My dataspace dataset',
    'hasPolicy': [{'@id': 'urn:odrl-policy:509d2b04-0f32-4795-92c5-8202f8066738',
      'permission': [{'action': 'use',
        'constraint': [{'rightOperand': 'user',
          'leftOperand': 'did:web:hola.es',
          'operator': 'eq'}]}],
      'obligation': [],
      '@type': 'Offer',
      'prohibition': []}],
    'distribution': [{'@type': 'Distribution',
      '@id': 'urn:distribu

In [89]:
print("Selected dataset target: \n{}\n".format(target_id))
print("Selected policy id: \n{}\n".format(policy_id))
print("Provider negotiation URL for DSP 2025: \n{}\n".format(data_space_provider_dsp_endpoint))


Selected dataset target: 
urn:dataset:b333014d-ee5b-435a-a065-3e0117ea35c0

Selected policy id: 
urn:odrl-policy:509d2b04-0f32-4795-92c5-8202f8066738

Provider negotiation URL for DSP 2025: 
http://127.0.0.1:1200/dsp/current



## Contract Negotiation

The process follows the negotiation protocol flow:
1. Request Init
2. Offer
3. Request
4. Acceptance
5. Verification
6. Finalization


In [ ]:
# Initialization of negotiation request (Consumer -> Provider)
url = data_space_consumer + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-request-init"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer + "/api/v1/negotiation-agent/dsp/current",
    "offer": {
        "@id": policy_id,  # policy id
        "target": target_id,  # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    cn_consumer_id = response_as_json["response"]["consumerPid"]
    cn_provider_id = response_as_json["response"]["providerPid"]
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

In [ ]:
# Provider creates initial offer (Provider -> Consumer)
url = data_space_provider + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,  # remove to test offer from provider
    "providerPid": cn_provider_id,  # remove to test offer from provider
    "offer": {
        "@id": "urn:policy-target:123",  # policy id
        "target": "urn:dataset:123",  # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

In [ ]:
# Consumer sends negotiation request based on offer (Consumer -> Provider)
url = data_space_consumer + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-request"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": "urn:policy-target:123",  # policy id
        "target": "urn:dataset:123",  # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

In [ ]:
# Provider updates/confirms the offer (Provider -> Consumer)
url = data_space_provider + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": "urn:policy-target:123",  # policy id
        "target": "urn:dataset:123",  # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

In [ ]:
# Consumer accepts the offer (Consumer -> Provider)
url = data_space_consumer + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-acceptance"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

In [ ]:
# Provider creates the Agreement (Provider -> Consumer)
url = data_space_provider + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-agreement"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

In [ ]:
# Consumer verifies the agreement (Consumer -> Provider)
url = data_space_consumer + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-verification"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

In [ ]:
# Provider finalizes the negotiation (Provider -> Consumer)
url = data_space_provider + "/api/v1/negotiation-agent/dsp/current/negotiations/rpc/setup-finalization"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    agreement_id = response_as_json["negotiationAgentModel"]["agreement"]["id"]
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

## Transfer Negotiation with Agents


In [ ]:
# 1. Transfer request setup (Consumer -> Provider)
url = data_space_consumer + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-request"
payload = {
    "associatedAgentPeer": "did:web:dataspace-agent-consumer.example.com",
    "agreementId": agreement_id,
    "format": "http+pull",
    "providerAddress": data_space_provider + "/api/v1/transfer-agent/dsp/current",
    "callbackAddress": data_space_consumer + "/api/v1/transfer-agent/dsp/current",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_process_consumer_pid = response_as_json["response"]["consumerPid"]
transfer_process_provider_pid = response_as_json["response"]["providerPid"]
print(f"Consumer PID: {transfer_process_consumer_pid}")
print(f"Provider PID: {transfer_process_provider_pid}")
print(f"Response:\n {json.dumps(response_as_json, indent=2)}")

In [ ]:
# 2. Start transfer (Provider -> Consumer)
url = data_space_provider + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": {
        "@type": "DataAddress",
        "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
        "endpoint": "http://example.com",
        "endpointProperties": [
            {
                "@type": "EndpointProperty",
                "name": "authorization",
                "value": "TOKEN-ABCDEFG"
            },
            {
                "@type": "EndpointProperty",
                "name": "authType",
                "value": "bearer"
            }
        ]
    },
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

In [ ]:
# 3. Suspend transfer (Consumer -> Provider)
url = data_space_consumer + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
# 4. Restart transfer (Consumer -> Provider)
url = data_space_consumer + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

In [ ]:
# 5. Suspension by Provider (Provider -> Consumer)
# (Note: Testing provider-initiated suspension)
url = data_space_provider + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
# Failure Test: Attempt start with invalid parameters
url = data_space_consumer + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": None
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
# Failure Test: Attempt duplicate or invalid suspension
url = data_space_consumer + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
# 6. Finalize transfer (Provider -> Consumer)
url = data_space_provider + "/api/v1/transfer-agent/dsp/current/transfers/rpc/setup-completion"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))